# Transfer Learning



## Pre trained network
![imagen](https://www.researchgate.net/publication/336874848/figure/fig1/AS:819325225144320@1572353764073/Illustrations-of-transfer-learning-a-neural-network-is-pretrained-on-ImageNet-and.png)

Estas son las arquitecturas de redes neuronales más utilizadas en la comunidad. Para más detalle sobre el funcionamiento de cada red, consultar el [Hands on Machine Learning for Python](https://learning.oreilly.com/library/view/hands-on-machine-learning/9781492032632/ch14.html#cnn_chapter).
* VGG-16
* VGG-19
* Inception V3
* XCeption
* ResNet-50

Las redes se pueden incorporar entrenadas, o sin entrenar.

Here are some key points about transfer learning with popular convolutional neural network architectures like VGG, Inception, XCeption, and ResNet:

VGG-16 and VGG-19 are classic CNN architectures developed by Visual Geometry Group at Oxford. They have very uniform architecture consisting of consecutive convolutional and max pooling layers. They are easy to adapt for transfer learning by replacing and retraining the classifier layers.

Inception V3 is Google's CNN architecture that introduced the inception module which allows for more efficient computation by using parallel convolutions of different sizes. It is more complex but can learn richer representations.

XCeption is an extension of Inception developed by Google that uses depthwise separable convolutions which are efficient and fast. It is a powerful model for transfer learning.

ResNet models like ResNet50 introduce skip connections which help with the vanishing gradient problem when training very deep networks. This allows them to train extremely deep networks efficiently. ResNet50 is one of the most commonly used models for transfer learning.

In general, these CNNs pretrained on large datasets like ImageNet provide rich feature representations that can be leveraged for computer vision transfer learning tasks. The tradeoff is between accuracy and computational efficiency.

Fine-tuning involves unfreezing later layers of the base model and retraining on new data. This allows adapting the pretrained features to new tasks. The early layers can be left frozen to retain the general features already learned.

Choosing the right model depends on your problem, data size, and computational constraints. But models like ResNet50, InceptionV3 and XCeption provide a great starting point in most cases.

## ResNet50V2

In [1]:
!pip install scikit-image
!pip install opencv-python

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
from skimage.io import imread
import cv2

from tensorflow.keras.applications.resnet_v2 import ResNet50V2, decode_predictions, preprocess_input

2024-02-26 19:20:51.260415: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
import ssl 
ssl._create_default_https_context = ssl._create_unverified_context      

In [5]:
# CODE MODEL

base_model = ResNet50V2(include_top= True, input_shape=(224, 224, 3), weights='imagenet', classifier_activation= 'softmax')


102869336/102869336 [==============================] - 13s 0us/step


In [6]:
len(base_model.layers) 

192

In [7]:
base_model.summary()

Model: "resnet50v2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_2[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 pool1_pad (ZeroPadding2D)   (None, 114, 114, 64)         0         ['conv1_conv[0][0]']          
                                                                                         

Cargamos algunas imagenes desde local, para ver qué tal funciona la red ResNet50V2 ya entrenada.

In [8]:
# CODE READ DATA

import os

"""
Reads image data from the given path and resizes each image to 224x224 pixels. 

Parameters:
path (str): The path to the image data directory.

Returns:
np.array: The array of resized image data.
"""

def read_data(path):
    X = []
    for file in os.listdir(path):
        image = imread(path + '/' + file)
        smallimage = cv2.resize(image, (224,224))
        print(path + '/' + file)

        X.append(smallimage)
    return np.array(X)


In [18]:
X_test = read_data('img')

#Preprocesar las imagenes tal y como entran en el model
X_test = preprocess_input(X_test)
print(X_test.shape)

img/dog.11857.jpg
img/cat.8037.jpg
img/horse.jpg
img/dog.11856.jpg
img/bear-1.jpg
img/cat.8016.jpg
img/pizza.jpg
img/karate.jpg
(8, 224, 224, 3)


In [10]:
# CODE preds

preds = base_model.predict(X_test)

1/1 [==============================] - 4s 4s/step


In [11]:
len(preds)

8

In [12]:
len(preds[0])

1000

In [19]:
decodes = decode_predictions(preds, top=1)

In [20]:
decodes

[[('n02106030', 'collie', 0.9952569)],
 [('n02124075', 'Egyptian_cat', 0.82098377)],
 [('n02113799', 'standard_poodle', 0.32455868)],
 [('n02106550', 'Rottweiler', 0.73290956)],
 [('n02132136', 'brown_bear', 0.9994439)],
 [('n02124075', 'Egyptian_cat', 0.6498506)],
 [('n07873807', 'pizza', 0.9971303)],
 [('n09835506', 'ballplayer', 0.54023355)]]

In [22]:
# CODE decode preds
for j in decodes:
    print('####################')
    for i,decode in enumerate(j):
        print("Predicted:\n", decode[1], decode[2])

####################
Predicted:
 collie 0.9952569
####################
Predicted:
 Egyptian_cat 0.82098377
####################
Predicted:
 standard_poodle 0.32455868
####################
Predicted:
 Rottweiler 0.73290956
####################
Predicted:
 brown_bear 0.9994439
####################
Predicted:
 Egyptian_cat 0.6498506
####################
Predicted:
 pizza 0.9971303
####################
Predicted:
 ballplayer 0.54023355


In [16]:
decodes = decode_predictions(preds, top=3)

In [28]:
decodes[6]

[('n07873807', 'pizza', 0.9971303)]

## VGG16
En este caso vamos a importar la red VGG16, que utilizaremos como red preentrenada y completaremos con una fully connected layer. 

In [33]:
TRAIN_PATH = '/Users/miguelopez/Desktop/Machine Learning_2/5-Deep_Learning/2-Redes_Convolucionales/data/train'

In [38]:
import pandas as pd
from sklearn.model_selection import train_test_split

IM_SIZE=32

filenames = os.listdir(TRAIN_PATH)
categories = []
for filename in filenames:
    category = filename.split('.')[0]
    categories.append(category)
    
df = pd.DataFrame({
    'filenames': filenames,
    'category': categories
})

train_df, validate_df = train_test_split(df,
                                         test_size=0.20,
                                         random_state=42)

train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)

In [39]:
train_df.head()

,filenames,category
0,cat.1088.jpg,cat
1,cat.1169.jpg,cat
2,cat.91.jpg,cat
3,dog.10489.jpg,dog
4,cat.485.jpg,cat


In [40]:
from keras.preprocessing.image import ImageDataGenerator


# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# Note that the validation data should not be augmented!
validation_datagen = ImageDataGenerator(rescale = 1.0/255. )

In [41]:
# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_dataframe(train_df,
                                                    TRAIN_PATH,
                                                    x_col='filenames',
                                                    y_col='category',
                                                    batch_size = 20,
                                                    class_mode = 'binary',
                                                    target_size = (IM_SIZE, IM_SIZE))

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = validation_datagen.flow_from_dataframe(validate_df,
                                                              TRAIN_PATH,
                                                              x_col='filenames',
                                                              y_col='category',
                                                              batch_size = 20,
                                                              class_mode = 'binary',
                                                              target_size = (IM_SIZE, IM_SIZE))

Found 5260 validated image filenames belonging to 2 classes.


Found 1316 validated image filenames belonging to 2 classes.


In [42]:
from tensorflow.keras.applications.vgg16 import VGG16

# CODE model
base_model = VGG16(input_shape=(IM_SIZE,IM_SIZE,3),
                    include_top=False,
                    weights = 'imagenet'
                )

58889256/58889256 [==============================] - 9s 0us/step


In [43]:
len(base_model.layers)

19

In [44]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 8, 8, 128)         0     

In [45]:

##### FULLY CONNECTED LAYER #####
# Flatten the output layer to 1 dimension
x = layers.Flatten()(base_model.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)

# Add a final sigmoid layer for classification
x = layers.Dense(1, activation='sigmoid')(x)

"""Compile and configure the model for training.

The base model input is used as the input layer. The predictions from the model 
are taken from the last dense layer after dropout and sigmoid activation.

The model is compiled with categorical cross entropy loss and accuracy metrics.
The Adam optimizer is used.
"""
model = tf.keras.models.Model(base_model.input, x)
    
model.compile(optimizer = 'adam', loss = 'binary_crossentropy',metrics = ['acc'])

In [46]:
len(model.layers)

23

In [47]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 8, 8, 128)         0     

Total params: 14977857 (57.14 MB)
This indicates the model has 14,977,857 total parameters, which takes up 57.14 megabytes of memory.

Trainable params: 14977857 (57.14 MB)
This shows that all of the 14,977,857 parameters are trainable. That is, their values will be updated during training to minimize the loss function.

Non-trainable params: 0 (0.00 Byte)
This shows the model has no non-trainable parameters. Some models like pretrained networks may have fixed, non-trainable params.

So in summary, this is a breakdown of the total number of parameters, the trainable parameters, and non-trainable parameters in the neural network model along with the memory footprint.

Having almost 15 million trainable parameters suggests this is likely a large, deep neural network architecture. The lack of any non-trainable parameters indicates all the weights are being learned from scratch during training rather than leveraging any pretrained weights.



 
Edit Last Message ⌘K


In [48]:
vgghist = model.fit(train_generator,
                    validation_data = validation_generator,
                    batch_size = 128,
                    epochs = 10)

Epoch 1/10
130/263 [=============>................] - ETA: 5:52 - loss: 0.6977 - acc: 0.5696

KeyboardInterrupt: 

### Inception

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

IM_SIZE = 75

train_generator = train_datagen.flow_from_dataframe(train_df,
                                                    TRAIN_PATH,
                                                    x_col='filenames',
                                                    y_col='category',
                                                    batch_size = 20,
                                                    class_mode = 'binary',
                                                    target_size = (IM_SIZE, IM_SIZE))

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = validation_datagen.flow_from_dataframe(validate_df,
                                                              TRAIN_PATH,
                                                              x_col='filenames',
                                                              y_col='category',
                                                              batch_size = 20,
                                                              class_mode = 'binary',
                                                              target_size = (IM_SIZE, IM_SIZE))



model.compile(optimizer = 'rmsprop',
             loss = 'binary_crossentropy',
             metrics = ['accuracy'])

Found 5260 validated image filenames belonging to 2 classes.
Found 1316 validated image filenames belonging to 2 classes.


In [ ]:
model.fit(train_generator,
         validation_data = validation_generator,
         steps_per_epoch = 100,
         epochs = 5)

Epoch 1/5
100/100 [==============================] - 23s 186ms/step - loss: 0.9782 - accuracy: 0.6540 - val_loss: 0.5831 - val_accuracy: 0.7272
Epoch 2/5
100/100 [==============================] - 20s 205ms/step - loss: 0.6786 - accuracy: 0.6850 - val_loss: 0.6164 - val_accuracy: 0.7363
Epoch 3/5
100/100 [==============================] - 18s 177ms/step - loss: 0.6064 - accuracy: 0.6950 - val_loss: 0.5667 - val_accuracy: 0.7606
Epoch 4/5
100/100 [==============================] - 19s 189ms/step - loss: 0.6065 - accuracy: 0.7125 - val_loss: 0.6184 - val_accuracy: 0.6991
Epoch 5/5
100/100 [==============================] - 19s 186ms/step - loss: 0.5676 - accuracy: 0.7385 - val_loss: 0.5456 - val_accuracy: 0.7470


## ResNet50V2 sin entrenar

In [ ]:
IM_SIZE=32

base_model = ResNet50V2(input_shape = (224, 224, 3),
                       include_top = False,
                       weights = None,
                       classifier_activation= "softmax")


train_generator = train_datagen.flow_from_dataframe(train_df,
                                                    TRAIN_PATH,
                                                    x_col='filenames',
                                                    y_col='category',
                                                    batch_size = 20,
                                                    class_mode = 'binary',
                                                    target_size = (IM_SIZE, IM_SIZE))

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = validation_datagen.flow_from_dataframe(validate_df,
                                                              TRAIN_PATH,
                                                              x_col='filenames',
                                                              y_col='category',
                                                              batch_size = 20,
                                                              class_mode = 'binary',
                                                              target_size = (IM_SIZE, IM_SIZE))

Found 5260 validated image filenames belonging to 2 classes.
Found 1316 validated image filenames belonging to 2 classes.


In [ ]:
len(model.layers)

23

In [51]:
x = layers.Flatten()(base_model.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)

# Add a final sigmoid layer for classification
x = layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(base_model.input, x)
    
model.compile(optimizer = 'adam', loss = 'binary_crossentropy',metrics = ['acc'])

In [53]:
model.fit(train_generator,
         validation_data = validation_generator,
         steps_per_epoch = 2,
         epochs = 5)

Epoch 1/5
2/2 [==============================] - 18s 15s/step - loss: 0.7461 - acc: 0.5000 - val_loss: 0.6971 - val_acc: 0.4065
Epoch 2/5
2/2 [==============================] - 17s 15s/step - loss: 0.6839 - acc: 0.5500 - val_loss: 0.7766 - val_acc: 0.4065
Epoch 3/5
2/2 [==============================] - 17s 15s/step - loss: 0.8726 - acc: 0.2250 - val_loss: 0.7088 - val_acc: 0.4065
Epoch 4/5
2/2 [==============================] - 18s 16s/step - loss: 0.6922 - acc: 0.5750 - val_loss: 0.6963 - val_acc: 0.5935
Epoch 5/5
2/2 [==============================] - 17s 15s/step - loss: 0.6739 - acc: 0.6500 - val_loss: 0.6982 - val_acc: 0.5935
